In [10]:
import folium
import numpy as np
from sampled_already import sampled_already
from long_lat_multifamily_la2 import longitude_latitude_dict
import pandas as pd
from clusterpanoref import cluster_to_pano_dict
from sklearn.neighbors import BallTree
import numpy as np


In [ ]:
csv_save_file = "panos.csv"
panos_df = pd.read_csv(csv_save_file)


# Get a list of all cluster keys from cluster_to_pano_dict
cluster_keys = list(cluster_to_pano_dict.keys())
print(f"Number of clusters: {len(cluster_keys)}")
print(f"First few cluster keys: {cluster_keys[:5]}")

In [ ]:


# Extract all cluster locations first
cluster_locations = []
cluster_keys_list = []

for cluster_key in cluster_keys:
    if cluster_key in cluster_to_pano_dict and len(cluster_to_pano_dict[cluster_key]) > 0:
        pano_id = cluster_to_pano_dict[cluster_key][0]
        pano_row = panos_df[panos_df['pano_id'] == pano_id]
        if not pano_row.empty:
            cluster_lat = pano_row['lat'].values[0]
            cluster_lon = pano_row['lon'].values[0]
            cluster_locations.append([cluster_lat, cluster_lon])
            cluster_keys_list.append(cluster_key)

# Convert to radians for BallTree with haversine metric
cluster_locations_rad = np.radians(cluster_locations)

# Create BallTree for fast spatial queries
print("creating ball")
tree = BallTree(cluster_locations_rad, metric='haversine')
print("created ball")
# Earth radius in meters
R = 6371000

counter = 0
for coords in longitude_latitude_dict.values():
#   if counter > 10:
#       break
  
  lat = coords[0]
  lon = coords[1]
  
  # Query the BallTree for clusters within 600m
  query_point = np.radians([[lat, lon]])
  # Convert 600m to radians for the query
  radius_in_radians = 600 / R
  
  # Find all points within radius
  indices = tree.query_radius(query_point, radius_in_radians)[0]
  
  nearby_clusters = []
  for idx in indices:
      cluster_key = cluster_keys_list[idx]
      # Calculate exact distance
      cluster_lat, cluster_lon = cluster_locations[idx]
      
      # Calculate distance in meters (approximate using Haversine formula)
      dlat = np.radians(cluster_lat - lat)
      dlon = np.radians(cluster_lon - lon)
      a = np.sin(dlat/2)**2 + np.cos(np.radians(lat)) * np.cos(np.radians(cluster_lat)) * np.sin(dlon/2)**2
      c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
      distance = R * c
      
      nearby_clusters.append((cluster_key, distance))
  
  print(f"Coordinates {lat}, {lon} have {len(nearby_clusters)} clusters within 600m")
  if nearby_clusters:
    print(f"Closest clusters: {sorted(nearby_clusters, key=lambda x: x[1])[:3]}")
  
  counter += 1
  print(counter)
  # Update sampled_already.py with the current coordinates
